In [ ]:
# TODO:
#   * set nodes' size proportional to their degree

In [ ]:
import numpy as np
import pandas as pd
import igraph as ig
import os
from scipy.stats import pearsonr

In [ ]:
import BraiAn

path_to_allen_json = "./data/AllenMouseBrainOntology.json"
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json)

In [ ]:
experiment = "soumnya"
group_1_animal_names = "C"
group_2_animal_names = "S"
marker = "CFos"
normalization = "Density"
comparison_folder = "CM-SM"

filename = f"{marker.lower()}_{normalization.lower()}.csv"
filename_path = os.path.join(".", "data", "experiments", experiment, "BraiAn_PLS_output", comparison_folder, filename)

In [ ]:
experiment = "soumnya"
group_1_animal_names = "C"
group_2_animal_names = "S"
marker = "CFos"
normalization = "RelativeDensity"
comparison_folder = "CF-SF"

filename = f"{marker.lower()}_{normalization.lower()}.csv"
filename_path = os.path.join(".", "data", "experiments", experiment, "BraiAn_PLS_output", comparison_folder, filename)

In [ ]:
group_1 = pd.read_csv(filename_path, sep=";", usecols=lambda c: group_1_animal_names in c or c == "BrainRegion")
assert "BrainRegion" in group_1.columns, f"'{filename}' is expected to have a column called 'BrainRegion'."
group_1 = group_1.set_index("BrainRegion")
group_1.reindex(index=group_1.index[::-1])
group_1_r = group_1.transpose().corr(method=lambda x,y: pearsonr(x,y)[0], min_periods=4)
group_1_p = group_1.transpose().corr(method=lambda x,y: pearsonr(x,y)[1], min_periods=4)

In [ ]:
group_2 = pd.read_csv(filename_path, sep=";", usecols=lambda c: group_2_animal_names in c or c == "BrainRegion")
assert "BrainRegion" in group_2.columns, f"'{filename}' is expected to have a column called 'BrainRegion'."
group_2 = group_2.set_index("BrainRegion")
group_2.reindex(index=group_2.index[::-1])
group_2_r = group_2.transpose().corr(method=lambda x,y: pearsonr(x,y)[0], min_periods=4)
group_2_p = group_2.transpose().corr(method=lambda x,y: pearsonr(x,y)[1], min_periods=4)

In [ ]:
p_cutoff = 0.05
r_cutoff = 0.9

fig = BraiAn.draw_chord_plot(group_2_r, group_2_p, r_cutoff, p_cutoff, AllenBrain,
                            ideograms_a=50,
                            title="A connectomics graph of Pearson correlation data",
                            size=1200,
                            no_background=False,
                            annotation1="This is the first annotation",
                            annotation2="This is the second annotation",
                            annotation3="This is the third annotation")
fig.show()

In [ ]:
fig.write_image("stress_M_chord_diagram.svg")